Notes: 
+ Utilize GPU
+ Also look at: https://huggingface.co/docs/transformers/model_doc/clip 

interesting predictions for these two first test of our image retreival system:

['f9ac806d818bfbd877c0345d614d4987', '97a1a3d6ac7a5bfaad8d1810996cbc38']

# Functions/Classes/Libs

In [ ]:
import json

In [ ]:
%%capture
!pip install --upgrade --no-cache-dir gdown

In [ ]:
!mkdir data
%cd data
%pwd

/content/data


'/content/data'

In [ ]:
# %%capture 
# if didn't download them, run each 'gdown' separately
# imgs_attrubutes, photos_id.csv, features.npy
!gdown --id '1aUx2SnevfQ4AZqfX6logwmlM-yHqfPZ3' 

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1aUx2SnevfQ4AZqfX6logwmlM-yHqfPZ3
To: /content/data/imgs_attributes.pickle
100% 5.98M/5.98M [00:00<00:00, 64.3MB/s]


In [ ]:
!gdown --id '1-4Of03ZTx1bZq6SsTbfyIeHet0Efw9ni'

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-4Of03ZTx1bZq6SsTbfyIeHet0Efw9ni
To: /content/data/photo_ids.csv
100% 3.75M/3.75M [00:00<00:00, 67.9MB/s]


In [ ]:
!gdown --id '1-1QZh1FBkMgnW4C88JjyRgwax2nj0sHV'

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-1QZh1FBkMgnW4C88JjyRgwax2nj0sHV
To: /content/data/features.npy
100% 116M/116M [00:00<00:00, 139MB/s] 


In [ ]:
# db.json, dialogue.json 
!gdown --id '1-R7Ie9ggco2xKf-Q-g6Bmase6bHMRnci'

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-R7Ie9ggco2xKf-Q-g6Bmase6bHMRnci
To: /content/data/db.json
100% 14.7M/14.7M [00:00<00:00, 175MB/s]


In [ ]:
!gdown --id '1-30wh_HisCn8vWPJSzEtwT8Vp1Om-fL-'

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-30wh_HisCn8vWPJSzEtwT8Vp1Om-fL-
To: /content/data/dialogues.json
100% 56.8M/56.8M [00:00<00:00, 256MB/s]


In [ ]:
import pickle 

with open('imgs_attributes.pickle', 'rb') as handle:
    imgs_attributes = pickle.load(handle)

In [ ]:
%cd ..

/content


In [ ]:
# importing CLIP Module and the right version of Pytorch
!pip install git+https://github.com/openai/CLIP.git
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 -f https://download.pytorch.org/whl/torch_stable.html

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-k_lkzwnv
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-k_lkzwnv
     |████████████████████████████████| 53 kB 1.3 MB/s 
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369221 sha256=1b2a4035d7d68b3f9591786a5c8ed5c38e2302dd168870cb9598c7d29cb8e620
  Stored in directory: /tmp/pip-ephem-wheel-cache-7a_niat4/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
Successfully built clip
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 735.4 MB 16 kB/s 
     |████████████████████████████████| 12.8 MB 20.7 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-

In [ ]:
import clip
import torch
# Load the open CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 130MiB/s]


In [ ]:
import pandas as pd
import numpy as np

# Load the photo IDs
photo_ids = pd.read_csv("data/photo_ids.csv")
photo_ids = list(photo_ids['photo_id'])

# Load the features vectors
photo_features = np.load("data/features.npy")

# Convert features to Tensors: Float32 on CPU and Float16 on GPU
if device == "cpu":
  photo_features = torch.from_numpy(photo_features).float().to(device)
else:
  photo_features = torch.from_numpy(photo_features).to(device)

# Print some statistics
print(f"Photos loaded: {len(photo_ids)}")

Photos loaded: 113560


In [ ]:
# Search sub-routines

def find_best_matches(text_features, photo_features, photo_ids, results_count=2):
  # Compute the similarity between the search query and each photo using the Cosine similarity
  similarities = (photo_features @ text_features.T).squeeze(1)

  # Sort the photos by their similarity score
  best_photo_idx = (-similarities).argsort()

  # Return the photo IDs of the best matches
  return [photo_ids[i] for i in best_photo_idx[:results_count]]



def encode_txt_search_query(search_query):
  with torch.no_grad():
    # Encode and normalize the search query using CLIP
    text_encoded = model.encode_text(clip.tokenize(search_query).to(device))
    text_encoded /= text_encoded.norm(dim=-1, keepdim=True)

  # Retrieve the feature vector
  return text_encoded


""" Since we have already encoded all images in dialogue file, we
do not need encode_image_search_query function. But in case you 
had out-of-dataset image and you wanted to encode that, use the following 
function """ 

# Function that computes the feature vectors for a batch of images (ids)
def encode_image_search_query(photos_batch):
  from PIL import Image
  # Load all the photos from the files
  photos = [Image.open(photo_file) for photo_file in photos_batch]
  
  # Preprocess all photos
  photos_preprocessed = torch.stack([preprocess(photo) for photo in photos]).to(device)

  with torch.no_grad():
      # Encode the photos batch to compute the feature vectors and normalize them
      photos_features = model.encode_image(photos_preprocessed)
      photos_features /= photos_features.norm(dim=-1, keepdim=True)

  # Transfer the feature vectors back to the CPU and convert to numpy
  return photos_features.cpu().numpy()


In [ ]:
# Search functions

def search_by_text_and_photo(query_text, query_photo_id, photo_weight=0.5):
  # Encode the search query
  text_features = encode_txt_search_query(query_text)

  # Find the feature vector for the specified photo ID
  query_photo_index = photo_ids.index(query_photo_id)
  query_photo_features = photo_features[query_photo_index]

  # Combine the test and photo queries and normalize again
  search_features = text_features + query_photo_features * photo_weight
  search_features /= search_features.norm(dim=-1, keepdim=True)

  # Find the best match
  best_photo_ids = find_best_matches(search_features, photo_features, photo_ids, 2)

  return best_photo_ids



def search_by_photo(query_photo_id,results_count=2):

  # Find the feature vector for the specified photo ID
  query_photo_index = photo_ids.index(query_photo_id)
  search_features = photo_features[query_photo_index]

  # make 1D tensor 2D
  search_features = search_features.resize_(1, search_features.shape[0])

  # print(search_features)
  # print('\n','shape:', search_features.shape)
  # return search_features
  # Find the best match
  best_photo_ids = find_best_matches(search_features, photo_features, photo_ids, results_count)

  return best_photo_ids



def search_by_txt(search_query, results_count=2):
  # Encode the search query
  text_features = encode_txt_search_query(search_query)
  print(text_features)
  print('\n','shape:', text_features.shape)
  # Find the best matches
  best_photo_ids = find_best_matches(text_features, photo_features, photo_ids, results_count)

  # Display the best photos
  for photo_id in best_photo_ids:
    print(photo_id)
    # display_photo(photo_id)




# Paly with CLIP !

In [ ]:
q = 'outdoor restaurant suitable for birthday party'
search_by_txt(q, results_count=2)

tensor([[ 4.3917e-04, -1.0178e-02,  1.2550e-02, -5.1575e-02,  3.6438e-02,
         -2.5162e-02, -1.1940e-02, -7.4951e-02,  1.9058e-02, -1.7014e-02,
          1.2268e-02,  1.0086e-02,  4.1924e-03, -7.5006e-04, -2.1095e-03,
          5.5809e-03,  8.8882e-03,  1.9363e-02,  7.8011e-03,  3.6335e-03,
         -1.1482e-03,  2.3773e-02, -1.8463e-02, -1.7929e-03,  2.7161e-02,
         -2.4323e-02, -3.7048e-02,  1.2230e-02,  5.9166e-03,  5.4665e-03,
         -6.4392e-02, -3.4180e-02,  5.1193e-03, -2.2491e-02, -5.5054e-02,
         -4.2755e-02,  3.7918e-03,  5.3368e-03, -2.6108e-02,  1.8768e-02,
          2.0123e-03,  4.1412e-02,  1.0887e-02,  1.3786e-02,  1.0689e-02,
         -1.3748e-02,  6.8604e-02, -9.0408e-03,  1.9928e-02, -6.1035e-03,
         -3.7292e-02, -8.2932e-03,  8.3771e-03, -4.0283e-02, -3.7140e-02,
          2.7390e-02, -3.2501e-02,  1.6022e-02, -1.2566e-02,  2.0386e-02,
         -2.4780e-02, -1.9135e-02,  4.4174e-03, -1.5091e-02, -3.8666e-02,
         -6.1111e-03,  7.1449e-03,  3.

# Driver code

## loading some files

In [ ]:
with open('data/dialogues.json') as f:
    dial = json.load(f)
print(type(dial))
# Output: dict
print(list(dial.keys())[:5])

<class 'dict'>
['105', '108', '109', '110', '112']


## User-side Evaluation

### Preparing X,Y test

In [ ]:
x_test , y_test = [], []

for dial_id in dial.keys(): 
  for turn in dial[dial_id]['dialogue']:
    sz = len(turn['user']['img_gts'])
    if (sz != 0):  
      for i in range(sz):
        x_test.append(turn['user']['imgs'][i][:-4])
        y_test.append(turn['user']['img_gts'][i])




In [ ]:
# # H1 : Are the labels of exact images different across dialogue.json and image_classes.json?

# i = -1
# for id in x_test:
#   i +=1 
#   if y_test[i] != imgs_attributes[id]['class']:
#     print('oh no man')

# # Corrolary: H1 hypothesis is rejected

In [ ]:
tst= '97a1a3d6ac7a5bfaad8d1810996cbc38'
search_by_photo(tst)

['97a1a3d6ac7a5bfaad8d1810996cbc38', '111fa03f33b252dd1b2cc12177ef8fa1']

### finding Yhat ( and excluding the possible EXACT matches) 

In [ ]:
y_hat = []
y_hat_img_ids = []
i = 0
for img_id in x_test:
  two_predictions =  search_by_photo(img_id)
  if two_predictions[0] == img_id:
    y_hat.append(imgs_attributes[two_predictions[1]]['class'])
    y_hat_img_ids.append(two_predictions[1])
  else:
    y_hat.append(imgs_attributes[two_predictions[0]]['class'])
    y_hat_img_ids.append(two_predictions[0])


### Calculating image match rate (accuracy of predicted 'concept' or label)

In [ ]:
sz = len(y_hat)
all = 0
for i in range(sz):
  if  y_hat[i] != '':
    all +=1



corrects = 0
for i in range(sz):
  if y_hat[i] == '':
    pass
  elif y_hat[i] == y_test[i]:
    corrects +=1
  else:
    print(i,'+' ,y_hat[i] ,'actual:\t', y_test[i])  

1 + coffee actual:	 drink
3 + nightlife actual:	 clarke quay
5 + night cityview actual:	 cityview at night
8 + coffee actual:	 drink
9 + ramen actual:	 egg with food
11 + seafood noodle actual:	 japanese noodle (ramen/udon/soba)
12 + lobby actual:	 store front
16 + coffee actual:	 drink
17 + coffee actual:	 drink
18 + dry noodle actual:	 meal
19 + night cityview actual:	 cityview at night
22 + lobby actual:	 store front
24 + night cityview actual:	 steel structure
25 + lobby actual:	 store front
26 + building front actual:	 store front
27 + building front actual:	 store front
29 + seafood noodle actual:	 japanese noodle (ramen/udon/soba)
30 + building front actual:	 store front
31 + night cityview actual:	 cityview at night
32 + building actual:	 building front
33 + seafood noodle actual:	 japanese noodle (ramen/udon/soba)
34 + grilled actual:	 set meal
38 + coffee actual:	 drink
39 + food actual:	 french fries
41 + ramen actual:	 egg with food
44 + dry noodle actual:	 meal
46 + grille

In [ ]:
all,sz

(2896, 3296)

In [ ]:
x_test[9], y_hat_img_ids[9]

('55f865656a0e403b017dce424c7971d1', '896ca0c0e9f78fa9c5492bbc59b5b68d')

In [ ]:
y_test[9], y_hat[9]

('egg with food', 'ramen')

In [ ]:
acc= corrects / all
acc

0.6906077348066298

## Agent side

### Preparing X,Y test sets

In [ ]:
agent_x_test , agent_y_test = [], []

for dial_id in dial.keys(): 
  for turn in dial[dial_id]['dialogue']:
    sz = len(turn['agent']['img_gts'])
    if (sz != 0):  
      for i in range(sz):
        agent_x_test.append(turn['agent']['imgs'][i][:-4])
        agent_y_test.append(turn['agent']['img_gts'][i])


### finding Yhat 

In [ ]:
agent_y_hat = []
agent_y_hat_img_ids = []
i = 0
for img_id in agent_x_test:
  two_predictions =  search_by_photo(img_id)
  if two_predictions[0] == img_id:
    agent_y_hat.append(imgs_attributes[two_predictions[1]]['class'])
    agent_y_hat_img_ids.append(two_predictions[1])
  else:
    agent_y_hat.append(imgs_attributes[two_predictions[0]]['class'])
    agent_y_hat_img_ids.append(two_predictions[0])


### Calculating image match rate

In [ ]:
agent_sz = len(agent_y_hat)
agent_all = 0
for i in range(sz):
  if  agent_y_hat[i] != '':
    agent_all +=1



agent_corrects = 0
for i in range(sz):
  if agent_y_hat[i] == '':
    pass
  elif agent_y_hat[i] == agent_y_test[i]:
    agent_corrects +=1
  else:
    print(i,'+' ,agent_y_hat[i] ,'actual:\t', agent_y_test[i]) 

4 + ramen actual:	 japanese noodle (ramen/udon/soba)
7 + steak actual:	 stewed dish
14 + steak actual:	 stewed dish
16 + coffee actual:	 cake/dessert
18 + soup actual:	 soup and noodle
23 + ramen actual:	 japanese noodle (ramen/udon/soba)
25 + french fries actual:	 prawn/tempura
26 + onion ring and chicken wing actual:	 twist dessert
27 + sandwich actual:	 western food
29 + hotel lobby actual:	 hotel bathroom
33 + set meal actual:	 rice set
34 + spring roll actual:	 sashimi
35 + seafood noodle actual:	 japanese noodle (ramen/udon/soba)
41 + night cityview actual:	 cityview
45 + street corner actual:	 clarke quay
46 + building interior actual:	 marina bay sands
47 + lounge actual:	 casino
48 + waterside cityview actual:	 poolside
49 + soup actual:	 soup and noodle
51 + claypot actual:	 soup
56 + satay actual:	 steak
59 + steak actual:	 stewed dish
64 + steak actual:	 grilled meat
68 + ramen actual:	 japanese noodle (ramen/udon/soba)
70 + soup actual:	 japanese noodle (ramen/udon/soba)
7

In [ ]:
agent_sz , agent_all

(6928, 2734)

In [ ]:
agent_acc= agent_corrects / agent_all
agent_acc

0.6327724945135332

In [ ]:
# total accuracy
(0.6327724945135332 * 2734 + 0.6906077348066298 * 2896 ) / (2734 + 2896 )

0.6608632326820604